In [5]:
import pandas as pd
import os

In [6]:
!mkdir '../data/txt_files/'
!mkdir '../data/tokenizer_folder/'

mkdir: cannot create directory ‘../data/txt_files/’: File exists
mkdir: cannot create directory ‘../data/tokenizer_folder/’: File exists


In [7]:
txt_files_dir = '../data/txt_files/'
tokenizer_folder = '../data/tokenizer_folder/'

In [8]:
train_df = pd.read_csv('../data/names.tsv.gz',compression='gzip',sep='\t')

In [9]:
train_df.columns

Index(['id', 'name'], dtype='object')

In [10]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
        # Create the filename using the prefix ID
        file_name = os.path.join(txt_files_dir, str(i)+'.txt')
        try:
            # Create the file and write the column text to it
            f = open(file_name, 'wb')
            f.write(row.encode('utf-8'))
            f.close()
        except Exception as e:  #catch exceptions(for eg. empty rows)
            print(row, e) 
        i+=1
    # Return the last ID
    return i
# Get the training data
data = train_df["name"]
# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0

# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)

# Removing the end of line character \n

nan 'float' object has no attribute 'encode'
nan 'float' object has no attribute 'encode'


In [11]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path


paths = [str(x) for x in Path(".").glob(txt_files_dir+"*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=10000, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])


In [12]:
#Save the Tokenizer to disk
tokenizer.save_model(tokenizer_folder)

['../data/tokenizer_folder/vocab.json', '../data/tokenizer_folder/merges.txt']

In [13]:
# Create the tokenizer using vocab.json and mrege.txt files

tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
# Test the tokenizer
# Show the tokens created
tokenizer.encode("test").tokens

['<s>', 'te', 'st', '</s>']